In [1]:
import os

In [2]:
input_path = '../code/original/guis'
eval_path = '../datasets/android/eval_set'

In [3]:
def clean_sequence(token_sequence):
    new_sequence = [ x for x in token_sequence if x is not '\n']
    sequence_no_comas = [ x for x in new_sequence if x is not ',']
    sequence_no_arrows = [ x for x in sequence_no_comas if x is not '{']
    sequence_no_arrows = [ x for x in sequence_no_arrows if x is not '}']
    del sequence_no_arrows[1]
    return sequence_no_arrows

In [16]:
total_sequences = []
total_ids = []
for f in os.listdir(input_path):
    gui = open("{}/{}".format(input_path, f), 'r')
    file_name = f[:f.find(".gui")]
    token_sequence = ['<START>']
    for line in gui:
        line = line.replace(",", " ").replace("\n", " \n")
        line = line.replace('row{', 'row')
        tokens = line.split(" ")
        for token in tokens:
            token_sequence.append(token)
#             print(token)
    token_sequence.append('<END>')
    # new dataset
    token_sequence = clean_sequence(token_sequence)
    total_sequences.append(token_sequence)
    total_ids.append(file_name)

In [25]:
eval_sequences = []
eval_ids = []
for f in os.listdir(eval_path):
    if f.find(".gui") != -1:
        gui = open("{}/{}".format(eval_path, f), 'r')
        file_name = f[:f.find(".gui")]
        token_sequence = ['<START>']
        for line in gui:
            line = line.replace(",", " ,").replace("\n", " \n")

            line = line.replace('row{', 'row')
            tokens = line.split(" ")
            for token in tokens:
                # self.voc.append(token)
                token.replace(',', '')
                token_sequence.append(token)
        token_sequence.append('<END>')
        # new dataset
        token_sequence = clean_sequence(token_sequence)
        eval_sequences.append(token_sequence)
        eval_ids.append(file_name)

In [26]:
total_sequences[0]
len(eval_sequences)

250

In [27]:
import nltk

In [28]:
def edit_distance(str1, str2, m, n): 
    # Create a table to store results of subproblems 
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)] 
  
    # Fill d[][] in bottom up manner 
    for i in range(m + 1): 
        for j in range(n + 1): 
  
            # If first string is empty, only option is to 
            # insert all characters of second string 
            if i == 0: 
                dp[i][j] = j    # Min. operations = j 
  
            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif j == 0: 
                dp[i][j] = i    # Min. operations = i 
  
            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif str1[i-1] == str2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # If last character are different, consider all 
            # possibilities and find minimum 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert 
                                   dp[i-1][j],        # Remove 
                                   dp[i-1][j-1])    # Replace 
  
    return dp[m][n] 

In [37]:
def wer(decode, target):
    """Computes the Word Error Rate (WER).
    WER is defined as the edit distance between the two provided sentences after
    tokenizing to words.
    Args:
    decode: string of the decoded output.
    target: a string for the ground truth label.
    Returns:
    A float number for the WER of the current decode-target pair."""
    
    # Map each word to a new char.
    words = set(decode + target)
    word2char = dict(zip(words, range(len(words))))
    
    new_decode = [chr(word2char[w]) for w in decode]
    new_target = [chr(word2char[w]) for w in target]
    
    final_decode = ''.join(new_decode)
    final_target = ''.join(new_target)
    return edit_distance(final_decode,final_target, len(final_decode), len(final_target))/len(final_target)

In [38]:
sum_wer = 0
sum_bleu = 0

for i in range(len(eval_sequences)):
    decodeid = total_ids.index(eval_ids[i])
    decode = total_sequences[decodeid]
    target = eval_sequences[i]
#     print(total_sequences[decodeid],eval_sequences[i])
    image_wer = wer(decode, target)
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([target], decode)
    #print(BLEUscore)
    sum_wer += image_wer
    sum_bleu += BLEUscore
total_wer = sum_wer / len(eval_sequences)
total_bleu = sum_bleu / len(eval_sequences)
total_wer, total_bleu


(0.5320588331717502, 0.30182939312256046)

In [ ]:
eval_ids[0]